In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import random


from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


In [9]:
def from_csv_to_lists(csv):
  """
    Esta función toma una archivo csv y los pasa a lista de tuplas.
    
    Args:
    - csv: archivo csv.

    Returns:
    - list(df.itertuples(index=False, name=None)): lisat de tuplas
    """
  #lee el csv y lo pasa  a dataframe
  df = pd.read_csv(csv)
  try:
    df = df.drop('Unnamed: 0', axis=1)
  except:
    pass
  #devuelve lisat de tuplas
  return list(df.itertuples(index=False, name=None))

In [10]:
#lista de prueba:
#lista_pelis = [("Drama","Movie","The Drop",2006,6,"tt0340084"),("Thriller","Short","Geocatching",2015,11,"tt5142104"),("Drama","Movie","Blade Runner 2049", 2017,10,"tt1856101"),("Mystery","Movie","The Fugitive",1993,9,"tt0106977")]

def datos_imdb(lista_pelis):
    """
    Esta función toma una lista de tuplas de peliculas (genero, tipo, nombre pelicula, año y mes de estreno, id pelicula), busca más informacion de la pelicula en imdb y crea una nueva lista de tuplas con toda la informacion incluida para cada pelicula, otra con las puntuaciones y otra con los actores.
    
    Args:
    - lista_pelis: lista de tuplas de peliculas (genero, tipo, nombre pelicula, año y mes de estreno, id pelicula) proviniente de la api MoviesDatabase.

    Returns:
    - lista_peliculas: lista de tuplas de cada pelicula con toda su informacion (id, nombre peli, tipo,	genero,	año, mes, director, actores, guionistas, argumento, duracion).
    - lista_imdb: lista de tuplas de cada peli con su puntuacion imdb (id, nombre pelicula, puntos imdb, año)
    - lista_tupla_actores: lista de tupla de los 10 actors principales de cada peli (id, actor1 - actor10)
    """

    lista_imdb =[]
    lista_tupla_actores=[]
    lista_peliculas = []
    lista_pelicula =  []

    driver = webdriver.Chrome()

    sleep(0.5)
    num = 1
    for tupla in lista_pelis[:1001]:
        driver.get("https://www.imdb.com/")

        driver.maximize_window()
        sleep(0.5)

        print(f'{num}. {tupla[2]}')
        try:
            #Escribe el título de la pelicula en el buscador
            driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)
        except:
            driver.get("https://www.imdb.com/")
            
            sleep(0.5)

            driver.find_element("css selector", "#suggestion-search").send_keys(tupla[-1], Keys.ENTER)
        
        sleep(0.5)
        # Puntución IMDB
        try:
            try:
                try:
                    imdb_puntos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
                except:
                    imdb_puntos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-3a4309f8-0.bjXIAP.sc-dffc6c81-1.ehUwiL > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK").text
            except:
                    imdb_puntos = driver.find_element("css selector", "div.sc-3a4309f8-0:nth-child(2) > div:nth-child(1) > div:nth-child(1) > a:nth-child(2) > span:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > span:nth-child(1)").text
        except:
            imdb_puntos = "NO DATA"
        print(f"puntos: {imdb_puntos}")

        # Director IMDB
        # está dando error al appendear: 
        # TypeError: 'builtin_function_or_method' object is not subscriptable
        try:
            try:
                try:
                    imdb_director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text

                except:
                    imdb_director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-10.hbBxmX > section > div.sc-dffc6c81-3.fhVOeP > div > ul > li:nth-child(1) > div > ul > li > a").text
            except:
                imdb_director = driver.find_element("css selector", ".sc-dffc6c81-3 > div:nth-child(1) > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2) > ul:nth-child(1) > li:nth-child(1) > a:nth-child(1)").text
        except:
            imdb_director = "NO DATA"

        print(f"director: {imdb_director}")
 
        # Guionistas IMDB
        guionistas = []
        for i in range(1,6):
            try:
                try:        
                    try: 
                        imdb_guion = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({i}) > a").text
                        guionistas.append(imdb_guion)
                    except:
                        imdb_guion = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-10.hbBxmX > section > div.sc-dffc6c81-3.fhVOeP > div > ul > li:nth-child(2) > div > ul > li:nth-child({i}) > a").text
                        guionistas.append(imdb_guion)
                except:
                    imdb_guion = driver.find_element("css selector", f".sc-dffc6c81-3 > div:nth-child(1) > ul:nth-child(1) > li:nth-child(2) > div:nth-child(2) > ul:nth-child(1) > li:nth-child({i}) > a:nth-child(1)").text
                    guionistas.append(imdb_guion)
            except:        
                break  
        print(f"guionistas: {guionistas}")
        
        # Argumento IMDB
        # Es importante dejar abrir la pagina en maximize porque si no no nos lo coge el css selector.
        try:
            try:
                try:
                    imdb_argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.ecgcFy > div.sc-e226b0e3-6.hfusNC > div.sc-e226b0e3-10.jcOwsU > section > p > span.sc-466bb6c-2.eVLpWt').text
                    
                except:
                    imdb_argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-4.dEqUUl > div.sc-e226b0e3-6.CUzkx > div.sc-e226b0e3-10.hbBxmX > section > p > span.sc-466bb6c-2.chnFO').text
            except:
                imdb_argumento = driver.find_element("css selector", '.sc-466bb6c-2').text
      
        except:    
            imdb_argumento = "NO DATA"    

        print(F"argunmento: {imdb_argumento}")
        # Minutos IMDB
        try:
                
            try:
                try:
                    imdb_minutos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(3)").text

                    lista_min = imdb_minutos.split()
                except:
                    imdb_minutos = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.dwkouE > div.sc-dffc6c81-0.grcyBP > ul > li:nth-child(3)").text
                    lista_min = imdb_minutos.split()
            except:
                    imdb_minutos = driver.find_element("css selector", "ul.ipc-inline-list:nth-child(2) > li:nth-child(2)").text
                    lista_min = imdb_minutos.split()
                
            print(lista_min)
            
            try: 
                #Ponemos un try...except porque nos hemos dado cuenta de que algunas películas tienen otra característica en la poscición de "hora,minutos"
                # Por ejemplo: 
                #__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(4)  
                
                if len(lista_min) > 1:
                    hora = int(lista_min[0].replace('h', ''))
                    hora 
                    min = int(lista_min[1].replace('min', ''))
                    duracion = 60*hora + min
                    print(duracion)

                elif len(lista_min) == 1:
                    if 'min' in lista_min[0]:
                        duracion= int(lista_min[0].replace('min',''))
                        print(duracion)   
                    elif 'h' in lista_min[0]: 
                        duracion= int(lista_min[0].replace('h',''))
                        duracion = 60*duracion 
                    print(duracion)
                else: duracion = lista_min[0]
            except:
                   duracion = "Formato min incorrecto"
        except:
            duracion = "NO DATA"            
        print(f"duracion: {duracion}")
        lista_actores = [tupla[-1]]
        for i in range(1, 11):
            # cogemos xpath porque el selector no nos cogía bien los nombres
            try:
                try:
                    try:
                        lista_actores.append(driver.find_element("xpath", f'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[3]/div[2]/div[2]/div[{i}]/div[2]/a').text)
                    
                    except:
                        lista_actores.append(driver.find_element("xpath", f'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[4]/div[2]/div[2]/div[{i}]/div[2]/a').text)
            
                except:
                    lista_actores.append(driver.find_element("xpath", f'/html/body/div[2]/main/div/section[1]/div/section/div/div[1]/section[2]/div[2]/div[2]/div[{i}]/div[2]/a').text)
            except:
                break
        print(f"actors: {lista_actores}")
        
        lista_tupla_actores.append(tuple(lista_actores))      
        #[("Drama","Movie","The Drop",2006,6,"tt0340084")
        #id	Nombre peli	tipo	genero	año	mes	director	director2	actor1	actorx	guionista1	guionista2	guionistas x	argumento	duracion

        #Lista de cada pelicula 
        lista_pelicula = [tupla[-1], tupla[2], tupla[1], tupla [0], tupla[3], tupla[4], imdb_director, lista_actores, guionistas, imdb_argumento, duracion]
        
        #lista  final todas las peliculas
        lista_peliculas.append(tuple(lista_pelicula))

        #lista puntuaciones imdb (id, nombre pelicula, puntos imdb, año)
        lista_imdb.append((tupla[-1], tupla[2], imdb_puntos, tupla[3]))

        #print(lista_imdb)

        num += 1
    driver.close()
    return lista_peliculas, lista_imdb, lista_tupla_actores

In [11]:
#archivo csv lista peliculas de la api a una lista de listas para correr el selenium
lista_csv_1991_10 = from_csv_to_lists('peliculas_1991.csv')

#pasamos lista a imdb y sacamos toda la informacion de imdb (datos peliculas, lista de actores y puntuacion imdb)
# ya para pasar a csv y bbdd --> lista_pelis --> id, nombre peli, tipo,	genero,	año, mes, director, actores, guionistas, argumento, duracion
# para pasar a rotten (google) --> lista_puntos_imdb --> id, nombre pelicula, puntos imdb, año
# para eliminar duplicados y pasar a imdb para buscar datos actores --> lista_actores_imdb --> (id, actor1, actor2, ..)

lista_peliculas_1991_10, lista_puntos_imdb_1991_10, lista_actores_imdb_1991_10 = datos_imdb(lista_csv_1991_10)

#pasar lista peliculas, lista actores, lista puntos imdb a csv
df_peliculas_1991_10 = pd.DataFrame(lista_peliculas_1991_10)
df_peliculas_1991_10.to_csv('lista_peliculas_1991_10.csv', index=False)
df_actores_1991_10 = pd.DataFrame(lista_actores_imdb_1991_10)
df_actores_1991_10.to_csv('actores_peliculas_1991_10.csv', index=False)
df_puntos_imdb_1991_10 = pd.DataFrame(lista_puntos_imdb_1991_10)
df_puntos_imdb_1991_10.to_csv('puntos_imdb_1991_10.csv', index=False)

5,2
Chin-Lai Sung
['Shi-Hsin Fu', 'Chin-Lai Sung']
85
['tt0077432', 'Alex Man', 'Emily Chu', 'Kiu-Wai Miu', 'Cheuk-Nin Cho', 'Yuk-Sing Ma', 'Tao Chiang', 'Shung-Fung Lau', 'Hsiang Lin Yin', 'Paul Chun', 'Mark Houghton']
[('tt0077432', 'Dip huet kei bing', '5,2', 1991)]
7,3
Dennis Christianson
['Dennis Christianson', 'Susan Killeen']
['tt0084015', 'Joe Moore', 'Elissa Dulce Hoopai', 'James Hong', 'Varoa Tiki', 'Megan Ward', 'Richard Vales', 'Danny Kamekona', 'Pat Morita', 'Dennis Chun', 'James Hutchinson']
[('tt0077432', 'Dip huet kei bing', '5,2', 1991), ('tt0084015', 'Goodbye Paradise', '7,3', 1991)]
7,3
Priyadarshan
['T. Damodaran']
140
['tt0092507', 'Mohanlal', 'Shankar', 'Ajayan Adoor', 'Mahesh Anand', 'Geetha', 'Cochin Hanifa', 'Jagadish', 'Ganesh Kumar', 'Manjula', 'Nandu']
[('tt0077432', 'Dip huet kei bing', '5,2', 1991), ('tt0084015', 'Goodbye Paradise', '7,3', 1991), ('tt0092507', 'Abhimanyu', '7,3', 1991)]
6,5
Marco Ferreri
['Liliane Betti', 'Marco Ferreri', 'Antonino Marino'